In [58]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geopandas as gpds

### Fazer Web Scrapping e pegar [ Valor, Endereço, Medida, Quarto e garagem ]

In [52]:
url = "https://www.zapimoveis.com.br/aluguel/apartamentos/pe+recife/?onde=,Pernambuco,Recife,,,,,city,BR%3EPernambuco%3ENULL%3ERecife,-8.05774,-34.882963,&transacao=Aluguel&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Apartamento&pagina="
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}

Imoveis = []


for x in range(1,11):
    site = requests.get(url + str(x), headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')
    container = soup.find("div", class_="listings__wrapper")
    lista = container.find_all("div", attrs={"class":"card-container"})


    for li in lista:
        valor = li.find('p', 'simple-card__price').find('strong').text
        valor = valor.replace('\n','').replace('/mês','')
            
        
        endereco = li.find('h2', 'simple-card__address').text
        endereco = endereco.replace('\n','')
        
        
        medida = li.find('div', 'simple-card__actions').find('ul').find('li','js-areas').text
        medida = medida.replace('area\n','').replace('\n','').replace('m²', '')
                
        
        quarto = li.find('div', 'simple-card__actions').find('ul').find('li','js-bedrooms').text
        quarto = quarto.replace('bedroom\n','').replace('\n','')
                    
        
        Imoveis.append([valor, endereco, medida, quarto])

df = df = pd.DataFrame(data=Imoveis, columns=['Aluguel','Endereco','medida','quarto'])
print(int(x))
df.head()

10


,Aluguel,Endereco,medida,quarto
0,R$ 2.000,"Rua Doutor Raul Lafayette, Boa Via...",60,2
1,R$ 2.000,"Rua Vicente Zírpoli, Caxangá ...",64,3
2,R$ 700,"Rua Presidente Nilo Peçanha, Imbir...",50,2
3,R$ 2.800,"Rua Isaac Salazar, Tamarineira ...",49,2
4,R$ 2.600,"Rua Conselheiro Portela, Espinheir...",60,2


### Converter medida para inteiro

In [53]:
df['medida'] = df['medida'].astype(int)
df.dtypes

Aluguel     object
Endereco    object
medida       int32
quarto      object
dtype: object

### Verificar se existem valores nulos no dataframe

In [54]:
df.isnull().sum()

Aluguel     0
Endereco    0
medida      0
quarto      0
dtype: int64

### Gerar localização a parti do endereço dos imóveis 

In [56]:
localizacao = gpds.tools.geocode(df['Endereco'], provider='nominatim', user_agent="imoveis")['geometry']
localizacao.head()

0    POINT (-34.90050 -8.12213)
1    POINT (-34.95265 -8.03094)
2    POINT (-34.91026 -8.11897)
3    POINT (-34.90551 -8.03255)
4    POINT (-34.89470 -8.04256)
Name: geometry, dtype: geometry

### Criar uma nova coluna e inserir os dados de localização

In [59]:
df['localizacao'] = localizacao
df.head()

,Aluguel,Endereco,medida,quarto,localizacao
0,R$ 2.000,"Rua Doutor Raul Lafayette, Boa Via...",60,2,POINT (-34.90050 -8.12213)
1,R$ 2.000,"Rua Vicente Zírpoli, Caxangá ...",64,3,POINT (-34.95265 -8.03094)
2,R$ 700,"Rua Presidente Nilo Peçanha, Imbir...",50,2,POINT (-34.91026 -8.11897)
3,R$ 2.800,"Rua Isaac Salazar, Tamarineira ...",49,2,POINT (-34.90551 -8.03255)
4,R$ 2.600,"Rua Conselheiro Portela, Espinheir...",60,2,POINT (-34.89470 -8.04256)


### Mostrar dataframe completo

In [60]:
display(df)

,Aluguel,Endereco,medida,quarto,localizacao
0,R$ 2.000,"Rua Doutor Raul Lafayette, Boa Via...",60,2,POINT (-34.90050 -8.12213)
1,R$ 2.000,"Rua Vicente Zírpoli, Caxangá ...",64,3,POINT (-34.95265 -8.03094)
2,R$ 700,"Rua Presidente Nilo Peçanha, Imbir...",50,2,POINT (-34.91026 -8.11897)
3,R$ 2.800,"Rua Isaac Salazar, Tamarineira ...",49,2,POINT (-34.90551 -8.03255)
4,R$ 2.600,"Rua Conselheiro Portela, Espinheir...",60,2,POINT (-34.89470 -8.04256)
...,...,...,...,...,...
335,R$ 3.200,"Rua Desembargador João Paes, Boa V...",61,2,POINT (-34.90984 -8.12666)
336,R$ 3.900,"Rua Antônio Falcão, Boa Viagem ...",55,2,POINT (-34.90037 -8.11322)
337,R$ 2.000,"Avenida Hélio Falcão, Boa Viagem ...",47,1,POINT (-34.90292 -8.12172)
338,R$ 4.000,"Boa Viagem, Recife",80,3,POINT (-34.90340 -8.12350)


### exportar o dataframe para o PowerBI

In [61]:
df.to_csv("Imoveis_340_Recife_ZapImoveis.csv", encoding='utf-8')